# Generate Lyapunov vector paths

This notebook generates paths of local maxima of the magnitude of the leading order Lyapunov vector. It uses persistent homology to locate the local maxima that eventually rise above a threshold, then generates the path of that local maximum in the image plane. It goes backward in time and generates a lead-in of some number of frames, maintains the path as long as the local maxima stays above the threshold, then also generates a tail of some number of frames after the local maximum dips below the threshold.

**ASSUMPTION:** Each component in the Lyapunov vector magnitude has only a single local maximum, and any that have more than one above the threshold are treated as separate. However, due to the way the persistence points are selected, only one might be selected. Additionally, any local maximum that wobbles in and out of the cutoff region generates a separate path for each time it enters the cutoff region.

In [59]:
import numpy as np
import scipy
from scipy.cluster import hierarchy as hc
import pandas as pd
from skimage import morphology
from scipy import misc
from matplotlib import pyplot as plt
import math, time
from skimage import measure
from scipy import signal
from scipy.ndimage.filters import gaussian_filter
from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn.apionly as sns 
from scipy import spatial

%matplotlib inline  

## Set up local environment variables

In [60]:
sDir = '/Users/birdbrain/Documents/Research/Projects/Schatz/Data/g21chaotic/r4000_2016_11_01_superfast'
centerx = 210 # Center-x of circular crop
centery = 210 # Center-y of circular crop
roll_width = 30 # Width of a single roll
crop_radius = 210-roll_width # Radius of circular crop
sLyapunovSupPD = 'g21per/pd_sub/%05d_sub_all.csv'
sBmp = 'g21per/bmps/%06d.bmp'
sLyapunovPaths = 'g21per/lyap_paths/%06d.txt'

buffer_frames = 20
lyap_death_threshold = 160
lyap_birth_threshold = 60

## Utility functions

In [62]:
def loadData(idx):
    # Get Lyapunov vector magnitude
    lyap_magnitude = misc.imread(sDir + "/" + (sBmp % idx))
    
    # Get Lyapunov superlevel set
    lyap_pd = pd.read_csv(sDir + "/" + (sLyapunovSupPD % idx))
    lyap_pd['idx'] = lyap_pd.index
    lyap_pd['path_id'] = 0
    lyap_pd['frame'] = idx

    return (lyap_magnitude, lyap_pd)

## Generate the paths

In [68]:
# Beginning and ending indices
start_index = 1000
end_index = 1030

# Distance cutoff radius
distance_cutoff = 5

# Initialize path counter
path_index = 1

# Working array for Lyapunov vectors and persistence diagrams
lyap_magnitudes = {}
lyap_pds = {}

# Load the first frames up to the number of buffer frames.
for index in range(start_index, start_index+buffer_frames):
    lyap_magnitudes[index], lyap_pds[index] = loadData(index)

# Build the paths out until you reach the end of the list
while index < end_index:
    
    # Match existing paths forward and eliminate those local maxima from the pool
    # Match to closest point that is within the cutoff radius
    
    # If path is active and the match forward doesn't fall in qualifying region in persistence plane, 
    # mark the path inactive. Otherwise, keep the path active.
    
    # If path is inactive, only match forward until the buffer has been exhausted or there is no more
    # local maximum within the distance cutoff radius

    # Check for new local maxima that qualify as the start of a path
    if index < (end_index - buffer_frames):
        
        # Get the local maxima corresponding to the qualifying region in 
        # the persistence plane that have not been eliminated
        current_pd = lyap_pds[index]
        new_paths = current_pd[(current_pd['death'] >= lyap_death_threshold) & \
                               (current_pd['birth'] <= lyap_birth_threshold) & \
                               (current_pd['path_id'] == 0)]
        
        # Loop through each new path
        for index, row in new_paths.iterrows():
            
            tmp_path = [index, 1, lyap_magnitudes[index][row['d_y'], row['d_x']], row['d_x'], row['d_y']]
            
            # Go backward in time for any new paths and back-fill the path from the buffer frames
            for backtrack in range(1,buffer_frames):
                tmpindex = index - backtrack
                backtrack_pd = lyap_pds[tmpindex]
                
                # Match to closest point that is within the cutoff radius
                prev_pts = np.vstack((backtrack_pd['d_x'], backtrack_pd['d_y']))
                prev_pts = prev_pts.T
                distmat = spatial.distance.pdist(np.hstack(([row['d_x'], row['d_y']], prev_pts)))
                distmat = spatial.distance.squareform(distmat)
                
                # Check for match
                if np.min(distmat[0,1:]) <= distance_cutoff:
                    matched_idx = np.argmin(distmat[0,1:])
                    
                    tmp_path = [tmpindex, 0, lyap_magnitudes[index][row['d_y'], row['d_x']], row['d_x'], row['d_y']]
                    
    
    # Increment index
    index = index + 1
    



(732, 12) (720, 12)
(1452, 12) (710, 12)
(2162, 12) (719, 12)
(2881, 12) (712, 12)
(3593, 12) (696, 12)
(4289, 12) (715, 12)
(5004, 12) (713, 12)
(5717, 12) (702, 12)
(6419, 12) (704, 12)
(7123, 12) (707, 12)
(7830, 12) (714, 12)
(8544, 12) (710, 12)
(9254, 12) (720, 12)
(9974, 12) (711, 12)
(10685, 12) (715, 12)
(11400, 12) (711, 12)
(12111, 12) (718, 12)
(12829, 12) (699, 12)
(13528, 12) (710, 12)
[   0    0   87  325    0    1   89  315    0    0    0 1000]


In [30]:
index = 1010

current_pd = lyap_pds[index]

In [31]:
new_paths = current_pd[(current_pd['death'] >= lyap_death_threshold) & (current_pd['birth'] <= lyap_birth_threshold)]

In [32]:
new_paths

,dim,birth,b_x,b_y,b_z,death,d_x,d_y,d_z
706,1,0,407,160,0,255,317,136,0


In [33]:
for idx, row in new_paths.iterrows():
    print row

dim        1
birth      0
b_x      407
b_y      160
b_z        0
death    255
d_x      317
d_y      136
d_z        0
Name: 706, dtype: int64


In [34]:
backtrack_pd = lyap_pds[index - 1]

In [42]:
prev_pts = np.vstack((backtrack_pd['d_x'], backtrack_pd['d_y']))
prev_pts = prev_pts.T

In [48]:
[row['d_x'], row['d_y']]

[317, 136]

In [49]:
distmat = spatial.distance.pdist(np.vstack(([row['d_x'], row['d_y']], prev_pts)))
distmat = spatial.distance.squareform(distmat)

In [57]:
print np.min(distmat[0,1:])
print prev_pts[np.argmin(distmat[0,1:])]

0.0
[317 136]
